# Analyse machine learning de fakes news

In [48]:
import re
import pandas as pd
import tensorflow as tf

## Import des données

In [49]:
# Chargement du fichier CSV de test et nettoyage sommaire
df_test = pd.read_csv("test.csv")

In [50]:
# Chargement du fichier CSV de train et nettoyage sommaire
df_train = pd.read_csv("train.csv")

## Nettoyage des données
- suppression des données vide

In [51]:
df_test['Text'] = df_test['Text'].apply(lambda x : str(x).strip())
df_test = df_test.dropna()

df_train['Text'] = df_train['Text'].apply(lambda x : str(x).strip())
df_train = df_train.dropna()

- suppression des entrées non anglaise

In [52]:
from langdetect import detect

def detect_lang(text):
    try:
        return detect(text)
    except:
        return 'unknown'

In [53]:
df_test['lang'] = df_test['Text'].apply(detect_lang)
df_test['lang'].unique()

df_train['lang'] = df_test['Text'].apply(detect_lang)
df_train['lang'].unique()

df_train = df_train[df_train['lang'] == 'en'] # Keep only english reviews
df_test = df_test[df_test['lang'] == 'en'] # Keep only english reviews

- nettoyage de la colonne de texte avec passage de tout le texte en lowercase

In [54]:
def clean_data(text):
    # Remove all the special characters
    processed_text = re.sub(r'\W', ' ', text)
    # remove all single characters
    processed_text= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_text)
    # Remove single characters from the start
    processed_text = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_text)
    # Substituting multiple spaces with single space
    processed_text = re.sub(r'\s+', ' ', processed_text, flags=re.I)
    # Removing prefixed 'b'
    processed_text = re.sub(r'^b\s+', '', processed_text)
    # Converting to Lowercase
    processed_text = processed_text.lower()

    return processed_text

In [55]:
df_train = df_train.drop(columns={'lang'})
df_test = df_test.drop(columns={'lang'})

df_train = df_train.drop_duplicates(subset="Text")
df_test = df_test.drop_duplicates(subset="Text")

df_train['Text'] = df_train['Text'].apply(clean_data)
df_test['Text'] = df_test['Text'].apply(clean_data)


## Affichage des données nettoyées
- données d'entraînements

In [56]:
df_train

,Labels,Text,Text_Tag
0,1,says the annies list political group supports ...,abortion
1,2,when did the decline of coal start it started ...,"energy,history,job-accomplishments"
2,3,hillary clinton agrees with john mccain by vot...,foreign-policy
3,1,health care reform legislation is likely to ma...,health-care
4,2,the economic turnaround started at the end of ...,"economy,jobs"
...,...,...,...
1262,1,says new report proves the stimulus supported ...,stimulus
1263,4,says its absurd to say theres enough data to e...,"health-care,science,sports"
1264,1,the wives of the 9 11 hijackers knew exactly w...,"history,terrorism"
1265,4,says jill mccabe supports late term abortions ...,abortion


- données de tests

In [57]:
df_test

,Text,Text_Tag
0,building wall on the s mexico border will take...,immigration
1,wisconsin is on pace to double the number of l...,jobs
2,says john mccain has done nothing to help the ...,"military,veterans,voting-record"
3,suzanne bonamici supports plan that will cut c...,"medicare,message-machine-2012,campaign-adverti..."
4,when asked by reporter whether hes at the cent...,"campaign-finance,legal-issues,campaign-adverti..."
...,...,...
1262,says his budget provides the highest state fun...,education
1263,ive been here almost every day,"civil-rights,crime,criminal-justice"
1264,in the early 1980s sen edward kennedy secretly...,"bipartisanship,congress,foreign-policy,history"
1265,says an epa permit languished under strickland...,"environment,government-efficiency"


## Analyse des données nettoyées 

In [59]:
# Nombre de mots uniques

def findWordsCount(column):
    # Créer un ensemble pour stocker les mots
    mots = set()

    # Parcourir les reviews et ajouter chaque mot à l'ensemble
    for review in column:
        for word in review.split():
            mots.add(word)
    return len(mots)

print(f"Nombre de mots uniques dans les données de test : {findWordsCount(df_test['Text'])}")
print(f"Nombre de mots uniques dans les données de train : {findWordsCount(df_train['Text'])}")

Nombre de mots uniques dans les données de test : 4364
Nombre de mots uniques dans les données de train : 4299
